In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Synthetic Data Generation using Gemini APIs

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/data-generation/synthetic_data_generation_using_gemini.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fdata-generation%2Fsynthetic_data_generation_using_gemini.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/data-generation/synthetic_data_generation_using_gemini.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/data-generation/synthetic_data_generation_using_gemini.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


| | |
|-|-|
|Author(s) | [Madhup Sukoon](https://github.com/vagrantism) |

## Overview

### Gemini

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro and Gemini Pro Vision models.

### Vertex AI Gemini API

The Vertex AI Gemini API provides a unified interface for interacting with Gemini models. There are two Gemini 1.0 Pro models available in the Gemini API:

- **Gemini 1.0 Pro model** (`gemini-1.0-pro`): Designed to handle natural language tasks, multi-turn text and code chat, and code generation.
- **Gemini 1.0 Pro Vision model** (`gemini-1.0-pro-vision`): Supports multimodal prompts. You can include text, images, and video in your prompt requests and get text or code responses.

You can interact with the Gemini API using the following methods:

- Use [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) for quick testing and command generation
- Use cURL commands
- Use the Vertex AI SDK

This notebook focuses on using the **Vertex AI SDK for Python** to call the Vertex AI Gemini API for the `gemini-1.0-pro` model.

For more information, see the [Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) documentation.

### Snowfakery
[Snowfakery](https://snowfakery.readthedocs.io/en/docs/index.html) is a framework for generating complex fake data at scale. It has a lot of in-built plugins, and allows you to extend its functionality by writing your own plugins.


### Objectives

This repository demonstrates how Gemini can be leveraged as a Snowfakery plugin for generating synthetic data based on a set of predefined schemas and data generation strategies. The framework is based on [Snowfakery](https://snowfakery.readthedocs.io/) which is itself based on [Faker](https://faker.readthedocs.io/). It requires the expected outputs to be codified in a YAML file per Snowfakery specs, detailing all the required fields and their respective data generation strategies. The framework currently supports 3 such strategies:

1. Static Values - can be included in the YAML file itself.
2. Faker based values - Leverages the Faker library to generate fake data.
3. LLM based values - Leverages an LLM (`gemini-1.0-pro`) call and a predefined prompt template to generate data

It is also possible to use arbitrary python functions to generate data and augment the pipeline. Interrelated schemas are also supported where the value of a given field depends on an already defined field, which allows us to create hierarchical data and complex schemas. The data generated via this framework is saved to a CSV file for further analysis / consumption.

Although this notebook can be used for any synthetic-data gneration use-case and schema, the current examples shows the simple use case of generating long (blogs) and short (comments) format contents using a given Wikipedia page as the seed data. While the primary purpose of the synthetic data generation pipeline is to generate data for testing, this can also be used to support tangential use-cases like running prompt experiments and comparisons at scale, building few-shot examples, evaluating fine-tuned models, etc.

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK for Python and other dependencies


In [2]:
! pip3 install --upgrade --user google-cloud-aiplatform snowfakery==3.6.2 Wikipedia-API==0.6.0

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [29]:
# Define project information
# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
PROJECT_ID = "gdc-ai-playground"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


In [3]:
import sys
import types
import logging
from typing import Any

import jinja2
import wikipediaapi
from vertexai.generative_models import GenerativeModel
from snowfakery.plugins import SnowfakeryPlugin

## Creating Plugins and Prompts

The following cells create the 2 custom plugins we need for this use case along with the needed prompts.

### Creating Plugins
The first plugin gives us the ability to interact with Wikipedia and fetch the contents for a given page. The second plugin allows us to interact with the Gemini API.

In [23]:
class SyntheticDataGeneration:
  """
  Implements all the extra functionality needed for this use-case
  """
  class Plugins(types.ModuleType):
    """
    Provides the plugins needed to extend Snowfakery
    """
    class Gemini(SnowfakeryPlugin):
        """
        Plugin for interacting with Gemini.
        """

        class Functions:
            """
            Functions to implement field / object level data generation
            """

            def fill_prompt(self, prompt_name: str | jinja2.Template, **kwargs) -> str:
                """
                Returns a formatted prompt
                """
                return (
                    jinja2.Environment(
                        loader=jinja2.FileSystemLoader(
                            searchpath="./"
                        )
                    )
                    .get_template(prompt_name)
                    .render(**kwargs)
                )

            def generate(
                self, prompt_name: str | jinja2.Template, temperature=0.9, top_p=1, **kwargs
            ) -> str | None:
                """
                A wrapper around Gemini plugin
                """
                logging.info("Preparing Prompt %s with %s", prompt_name, kwargs)
                prompt = self.fill_prompt(prompt_name, **kwargs)
                logging.info("Prompt %s Prepared", prompt_name)
                try:
                    logging.info("Calling Gemini For %s", prompt_name)
                    response = GenerativeModel("gemini-1.0-pro-001").generate_content(
                        [prompt],
                        generation_config={
                            "max_output_tokens": 8192,
                            "temperature": temperature,
                            "top_p": top_p,
                        },
                    )
                except Exception as e:
                    logging.error(
                        (
                            "Unable to generate text using %s.\n"
                            "Prepared Prompt: \n%s\n\nError: %s"
                        ),
                        prompt_name,
                        prompt,
                        e,
                    )
                    return None

                try:
                    return response.text
                except Exception as e:
                    logging.error(
                        (
                            "Unable to generate text using %s.\n"
                            "Prepared Prompt: \n%s\n\n"
                            "Received Response: \n%s\n\n"
                            "Error: %s"
                        ),
                        prompt_name,
                        prompt,
                        response,
                        e,
                    )
                    return None

    class Wikipedia(SnowfakeryPlugin):
        """
        Plugin for interacting with Wikipedia.
        """

        class Functions:
            """
            Implements a single function to fetch a Wikipedia page
            """

            def get_page(self, title: str):
                """
                Returns the title, URL and sections of the given wikipedia page
                """
                logging.info("Parsing Wikipedia Page %s", title)
                page = wikipediaapi.Wikipedia("Snowfakery (example@google.com)", "en").page(
                    title
                )
                results = {
                    "sections" : {},
                    "title": page.title,
                    "url": page.fullurl
                }
                sections = [(s.title, s) for s in page.sections]
                while sections:
                    sec_title, sec_obj = sections.pop()
                    if sec_title not in [
                        "External links",
                        "References",
                        "See also",
                        "Further reading",
                    ]:
                        if sec_obj.text:
                            results['sections'][sec_title] = sec_obj.text
                        for sub_sec in sec_obj.sections:
                            sections.append((f"{sec_title} - {sub_sec.title}", sub_sec))
                logging.info("Parsing Wikipedia Page %s Complete", title)
                return results

### Making plugins discoverable

We add the created class to `sys.modules` to ensure Snowfakery can find them and import them as modules as needed.

In [24]:
sys.modules["SyntheticDataGeneration.Plugins"] = SyntheticDataGeneration.Plugins(name="SyntheticDataGeneration.Plugins")

### Creating Prompt Templates

We add the created class to `sys.modules` to ensure Snowfakery can find them and import them as modules as needed.

In [25]:
%%writefile blog_generator.jinja
You are an expert content creator who writes detailed, factual blogs.
You have been asked to write a blog about {{idea_title}}.
To get you started, you have also been given the following context about the topic:

{{idea_body}}

Ensure the blog that you write is interesting,detailed and factual.
Take a deep breath and start writing:

Overwriting blog_generator.jinja


In [26]:
%%writefile comment_generator.jinja
You are {{first_name}} {{last_name}}. You are {{age}} years old. You are interested in {{interests}}. You work at {{organization}} as a {{profession}}.
You came across the following article:

{{blog_title}}

{{blog_body}}

Present your thoughts and feelings about the article in a short comment.

Comment:

Overwriting comment_generator.jinja


## Creating the Recipe
In order to generate synthetic data, the schema of the synthetic data must be defined first. This is done by creating a `recipe` in a YAML format as demonstrated below, more details on writing recipes can be found [here](https://snowfakery.readthedocs.io/en/latest/#central-concepts).

In [36]:
recipe = """
- plugin: SyntheticDataGeneration.Plugins.Wikipedia
- plugin: SyntheticDataGeneration.Plugins.Gemini
- option: wiki_title
- var: __seed
  value:
    - Wikipedia.get_page :
      title : ${{wiki_title}}

- object : users
  count : ${{random_number(min=100, max=500)}}
  fields :
    first_name : ${{fake.FirstName}}
    last_name : ${{fake.FirstName}}
    age:
      random_number:
        min: 18
        max: 95
    email : ${{fake.Email}}
    phone : ${{fake.PhoneNumber}}
    interests : ${{fake.Bs}}
    postal_code : ${{fake.Postalcode}}
    organization : ${{fake.Company}}
    profession : ${{fake.Job}}

- object : seeds
  fields :
    title : ${{__seed['title']}}
    url : ${{__seed['url']}}
    section_count : ${{__seed['sections'] | length}}

  friends:
    - object : blog_ideas
      count : ${{seeds.section_count}}
      fields :
        seed_id : ${{seeds.id}}
        section : ${{(__seed.sections.keys() | list)[child_index]}}
        body : ${{__seed.sections[section]}}

      friends:
        - object : blog_posts
          fields :
            blog_idea_id : ${{blog_ideas.id}}
            title : ${{seeds.title}} - ${{blog_ideas.section}}
            body :
              - Gemini.generate:
                prompt_name : blog_generator.jinja
                idea_title : ${{title}}
                idea_body : ${{blog_ideas.body}}
            author : Gemini

          friends:
            - object : blog_post_comments
              fields :
                blog_post_id : ${{blog_posts.id}}
                author_id :
                  random_reference : users
                author_email : ${{author_id.email}}
                comment :
                  - Gemini.generate:
                    prompt_name : comment_generator.jinja
                    first_name : ${{author_id.first_name}}
                    last_name : ${{author_id.last_name}}
                    age : ${{author_id.age}}
                    interests : ${{author_id.interests}}
                    organization : ${{author_id.organization}}
                    profession : ${{author_id.profession}}
                    blog_title : ${{blog_posts.title}}
                    blog_body : ${{blog_posts.body | truncate(1000)}}
"""

## Generating Data

In [37]:
from io import StringIO
from snowfakery import generate_data

generate_data(
    StringIO(recipe),
    output_format="csv",
    output_folder="outputs",
    user_options={"wiki_title": "Python_(programming_language)"},
)

Created outputs/users.csv
Created outputs/blog_post_comments.csv
Created outputs/blog_posts.csv
Created outputs/blog_ideas.csv
Created outputs/seeds.csv
Created outputs/csvw_metadata.json


### Results

The synthetic data has been generated and stored as CSV files in the `outputs` folder.